In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

diabetes_df = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

1. First, load the dataset and do some basic EDA.

In [ ]:
diabetes_df.info()

In [ ]:
diabetes_df.shape

In [ ]:
diabetes_df.head(10)

In [ ]:
diabetes_df.describe()

In [ ]:
diabetes_df.isnull().sum()

In [ ]:
diabetes_df.nunique()

In [ ]:
diabetes_df.groupby(diabetes_df['Outcome']).count()

2. Detect the problems with the data, such as missing values and outliers. Are there any nonsense values that you need to handle? For the missing values, think about which techniques would be the most suitable for handling these values. Regarding the outliers, think about their potential effects on your analysis and select an appropriate method to deal with them.


In [ ]:
plt.figure(figsize=(18,10))

var_list = diabetes_df.columns.unique()

for index, column in enumerate(var_list):
  plt.subplot(3,3,index+1)
  plt.hist(diabetes_df[column], bins=10)
  plt.title('Distribution of {}'.format(column))

plt.tight_layout()
plt.show()

Not possible for skin thickness, insulin, bmi, or blood pressure to be 0. Those people would be dead. assume incorrect values. fill with mean or median.

In [ ]:
# variables to replace 0 values with mean values
fill_list = ['SkinThickness', 'Insulin', 'BMI', 'BloodPressure']

for i in fill_list:
  diabetes_df[i].replace(0, diabetes_df[i].mean(), inplace=True)

In [ ]:
diabetes_df.describe()

In [ ]:
# plots after filling 0 values with mean
plt.figure(figsize=(18,10))

var_list = diabetes_df.columns.unique()

for index, column in enumerate(var_list):
  plt.subplot(3,3,index+1)
  plt.hist(diabetes_df[column], bins=10)
  plt.title('Distribution of {}'.format(column))

plt.tight_layout()
plt.show()

In [ ]:
#  check for normality 
print(stats.describe(diabetes_df['Glucose']))
print(stats.describe(diabetes_df['BloodPressure']))

In [ ]:
# considered winsorizing the data to handle outliers
#from scipy.stats.mstats import winsorize
#diabetes_df['winsorized_preg'] = winsorize(diabetes_df['pregnancies'], (0, .15))
#diabetes_df['winsorized_skin'] = winsorize(diabetes_df['skin_thickness'], (0, .15))
#diabetes_df['winsorized_insulin'] = winsorize(diabetes_df['insulin'], (0, .15))
#diabetes_df['winsorized_bp'] = winsorize(diabetes_df['blood_pressure'], (0, .15))
#diabetes_df['winsorized_bmi'] = winsorize(diabetes_df['bmi'], (0, .15))

3. Explore the data using univariate and multivariate exploration techniques. Be sure to pay special attention to your target variable. In this regard, focus on finding the relevant variables that may affect the likelihood of having diabetes.

In [ ]:
#univar analysis
plt.figure(figsize=(10,8))
sns.pairplot(data=diabetes_df, hue='Outcome')

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(diabetes_df.corr(), cbar=True, cmap='coolwarm', annot=True)

In [ ]:
# correlation of variables with outcome
corr_with_outcome = (diabetes_df.corrwith(diabetes_df['Outcome'])).sort_values(ascending=False)
print(corr_with_outcome)

In [ ]:
diabetes_df.var()

features: age, glucose, bmi, skin thickness

4. In the feature-engineering step, you need to select a suite of variables that you think would be ideal in the modeling phase. More concretely, you may discard some variables that are highly correlated with the other ones or the variables that you think are irrelevant for predicting whether a patient has diabetes.

In [ ]:
diabetes_df = diabetes_df.dropna()

X= diabetes_df[['Glucose', 'Insulin', 'Age', 'SkinThickness', 'BMI', 'Pregnancies']]

X = StandardScaler().fit_transform(diabetes_df)

Xt = X.T
Cx = np.cov(Xt)
print('Covariance Matrix:\n', Cx)

In [ ]:
eig_val_cov, eig_vec_cov = np.linalg.eig(Cx)

for i in range(len(eig_val_cov)):
  eigvec_cov = eig_vec_cov[:, i].T
  print('Eigenvector {}: \n{}'.format(i+1, eigvec_cov))
  print('Eigenvalue {}: {}'.format(i+1, eig_val_cov[i]))
  print(40*'-')

  print('Proportion of total variance', eig_val_cov/sum(eig_val_cov))

In [ ]:
plt.plot(eig_val_cov)
plt.title('Diabetes PCA Scree Plot')
plt.show()

print(eig_val_cov)

In [ ]:
X = diabetes_df

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
diabetes_df['pca1'] = sklearn_pca.fit_transform(X)

print('percentage of total variance in data set explained by each component in Sklearn PCA.', sklearn_pca.explained_variance_ratio_)

In [ ]:
diabetes_df.corrwith(diabetes_df['Outcome']).sort_values(ascending=False)